## Subject finder using SpaCy

When exploring a long text, or a series of texts with overlapping characters (like *Star Wars* novels), it may be useful to look at the verbs that co-occur with characters. What are the characters in your text *doing*, and how do those actions differ between characters?

Ideally, you'd want to do some [coreference resolution](https://towardsdatascience.com/most-popular-coreference-resolution-frameworks-574ba8a8cc2d) to be able to link all the pronouns in the text to the character they refer to. Unfortunately, there's no easy or reliable way to go about doing it. Only using character names, and ignoring pronouns, is much more feasible.

This code takes a folder of plain-text (.txt) files and creates, in the same directory as that folder, a CSV file with each subject, verb, and the name of the text file it came from.

### Step 0. Download SpaCy and language model
If you don't already have the SpaCy library installed, the code below downloads it along with the English language model. This notebook should work for any [language with a SpaCy model](https://spacy.io/models). Just substitute the name of the model for *en_core_web_sm* (For instance, if you wanted to use Lithuanian, you can replace `en_core_web_sm` with `lt_core_news_sm`). Places where you need to do this substitution are commented in the code.

In [ ]:
import sys
!{sys.executable} -m pip install spacy

In [ ]:
#Replace en_core_web_sm with another model name here for other languages
!{sys.executable} -m spacy download en_core_web_sm

### Step 1. Import modules
To use a [SpaCy language model](https://spacy.io/models) other than English, replace `en_core_web_sm` with the model name in the cell below.

In [ ]:
# os is used for navigating directories
import os
# spacy is used for identifying the subjects and verbs
import spacy
#Replace en_core_web_sm with another model name here for other languages
import en_core_web_sm
#Replace en_core_web_sm with another model name here for other languages
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp('She really liked Karen and Andrew, and she couldn\'t believe Kristy is doing this.')
#doc = nlp('Mary Anne Looks at Kristy in disbelief')
from spacy import displacy
displacy.render(doc, style="dep")

### Step 2. Select directory
Edit the. code cell below to put in the full path to the directory/folder with the text files (.txt) you want to work with. Note that this notebook will process *all* text files in the directory you specify, so you may need to put just the files you're interested in into their own directory.

If you have only one text file (e.g. a single novel) and aren't comfortable reworking the code to run on only one file, you can create a directory and put the text file inside it.

Here's the syntax to specify the full path to a folder called *YOUR-FOLDER* within the Documents directory:

* On Mac: '/Users/YOUR-USER-NAME/Documents/YOUR-FOLDER'
* On Windows: 'C:\\Users\\YOUR-USER-NAME\\Documents\\YOUR-FOLDER'

In [ ]:
directory = '/Users/qad/Documents/dhtrek/dhtrek_textcorpus'

In [ ]:
#Changes the notebook's working directory to the directory you specified
os.chdir(directory)

### Step 3. Find subjects
The code cells below reads each text file in the directory, finds every subject and verb, and writes it to a CSV file along with the filename it's from. The files are processed alphabetically, so that if something breaks (e.g. text files bigger than 1 MB may trigger a memory error), you can figure out what files are left to be done.

By default, the CSV file is called `charverbs.csv` and gets created inside the directory with the text files. You can give the file a different name in the code cell below, but keeping `.csv` is recommended.

Doing the NLP parse can be time-consuming, particularly if you have a large number of files. If your text is on the scale of hundreds of novels, expect it to take hours.

In [ ]:
charverbfile = 'trek-verbs-lemma.csv'

In [ ]:
#Opens the output file
with open(charverbfile, 'w') as out:
    out.write('Filename, Subject, Verb' + '\n')
    #Sorts the files alphabetically
    for filename in sorted(os.listdir(directory)):
        #Looks for .txt files
        if filename.endswith('.txt'):
            #Opens each file
            with open(filename, 'r') as bookfile:
                #Reads in the text in the file
                book = bookfile.read()
                #NLP parse of the text
                doc = nlp(book)
                #Noun chunks are the part of the SpaCy dependency parse that we need
                for chunk in doc.noun_chunks:
                    #If the dependency relation is 'nsubj' (noun subject)
                    if chunk.root.dep_ == 'nsubj':
                        #Write the filename, the noun chunk, the verb, and then a newline character
                        strsubj = str(chunk.text)
                        cleansubj = strsubj.replace(',', '')                        
                        out.write(filename + ', ' + cleansubj + ', ' + chunk.root.head.text + '\n')

## Interlude
Now you have a text file with a lot of subjects and verbs -- most of which you may not be able to use (because they involve a pronoun that you can't easily resolve to a character, or don't refer to a character at all). 

You may want to just explore this file in an interface that makes it easier to read with your eyeballs. You could load it into [OpenRefine](https://openrefine.org/) and use the *Text Filter* function on the Subject or Verb column to see what verbs are associated with a character, or which characters are associated with a verb. (If your corpus is small enough, you might be able to use *Text Facet* on the verb column to see what all the verbs are.) This can be a useful step to surface potential problems you might encounter as you run the next set of code; for instance, you could discover that searching for the character name "Mara" (with no space at the end) also gets you "Marauders" as a subject.

What I did next was create a copy of the `charverbs.csv` file, and removed all rows with a pronoun in them, using [regular expressions](https://programminghistorian.org/en/lessons/understanding-regular-expressions) in a plain text editor. (E.g. to get rid of "she" as a pronoun, I searched for `^.*, she(.*)` and replaced those lines with nothing.)

Once I got rid of all the pronouns, I went back to Python with the resulting CSV (saved in the folder of source texts, just like the original output CSV) to look for the most common subjects (which should largely align with the main characters).

### Step 4. Load CSV into a dataframe
Pandas dataframes are basically the spreadsheets of Python. They offer numerous options for creating complex subsets from large amounts of tabular data. The code cells below import the pandas module and create a dataframe from the CSV where you've removed the pronouns.

The code below assumes that you've run the code cells above, and that your working directory is the same directory you specified earlier. If you've named your clean CSV something other than `charverbs-clean.csv`, specify it in the second code cell below.

In [ ]:
import pandas as pd

In [ ]:
charverbs = pd.read_csv('charverbs-clean.csv')

In [ ]:
charverbs